In [11]:
import numpy as np
import pandas as pd
import requests
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim
print('Libraries imported')

ModuleNotFoundError: No module named 'geocoder'

In [18]:
!pip install bs4

     |████████████████████████████████| 122kB 5.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


#### Extract the content from the link and specify to extract the first item (table) and save it as you want

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]
print(df)
df.to_csv('toronto_data.csv')

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

#### Read and show csv file

In [3]:
df_toronto = pd.read_csv('toronto_data.csv')

In [4]:
df_toronto.head()

,Unnamed: 0,Postal Code,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Drop Unnamed: 0 column and clear all rows which are assigned a "Not Assigned" value in Borough column

In [5]:
df_toronto = df_toronto.drop(['Unnamed: 0'], axis=1)

In [6]:
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']

In [7]:
df_toronto = df_toronto.reset_index()

In [8]:
df_toronto = df_toronto.drop(['index'], axis=1)

In [9]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Import Geocoder, define a function called get_geocoder creating a loop to retrieve all coordinates related to Toronto area postal codes and assig coordinates to each row by postal code to the dataframe

In [13]:
import geocoder

def get_geocoder(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g=geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

for i in range (0,len(df_toronto['Postal Code'])):
    df_toronto.iloc[i]['Latitude'],df_toronto.iloc[i]['Longitude']=get_geocoder(df_toronto.iloc[i]['Postal Code'])

In [15]:
df_toronto['Latitude'], df_toronto['Longitude'] = zip(*df_toronto['Postal Code'].apply(get_geocoder))

In [38]:
df_toronto.loc[df_toronto['Borough'] == 'North York'].head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
7,M3B,North York,Don Mills,43.74923,-79.36186
10,M6B,North York,Glencairn,43.70687,-79.44812


#### Import KMeans from Scikitlearn library, BeautifulSoup and Requests

In [19]:
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup as bs
import requests

### TORONTO MAP
#### Obtain lat and lng coordinates for Toronto and display map with neighbourhoods

In [43]:
address = ['Toronto, Canada']
geolocator = Nominatim(user_agent = 'foursquare_agent')
location = geolocator.geocode(address)
lat=location.latitude
lng=location.longitude
print(lat, lng)

43.6534817 -79.3839347


In [58]:
toronto_lat = lat
toronto_lng = lng
toronto_map = folium.Map(location=[toronto_lat,toronto_lng], zoom_start=10)
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#195e83',
        fill_opacity=0.5,).add_to(toronto_map)
toronto_map.save('toronto_map.png')
toronto_map

### Downtown Toronto borough chosen for assignment
#### Get credentials from Foursquare API

In [30]:
CLIENT_ID = '0WP5KFQ5LRU2PFDNWB05LIATWK41Q5YHYATMUFXHA5011A0K'
CLIENT_SECRET = 'EHOXG2GKTFFDNNC3HFB3QUDQHJ53NW2HZX2BQH2QRAYJ3PW3'
VERSION = '20190530'

#### Get dataframe only for North York borough

In [92]:
downtown_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head(30)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
5,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
6,M6G,Downtown Toronto,Christie,43.66869,-79.42071
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.64970,-79.38258
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.64285,-79.38076
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.64710,-79.38153


#### Obtain lat and lng coordinates for North York to center map and display according to these coordinates with its respective neighbourhoods

In [91]:
address = ['Downtown Toronto, Toronto, Canada']
geolocator = Nominatim(user_agent = 'foursquare_agent')
location = geolocator.geocode(address)
lat=location.latitude
lng=location.longitude
print(lat, lng)

43.6563221 -79.3809161


In [93]:
downtown_lat = lat
downtown_lng = lng

map_downtown = folium.Map(location=[downtown_lat, downtown_lng], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Borough'], downtown_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)
    
map_downtown

### Get top 100 venues in Neighbourhood Glencairn from North York

In [94]:
neighbourhood_lat = downtown_data.loc[5, 'Latitude'] 
neighbourhood_lng = downtown_data.loc[5, 'Longitude']
neighbourhood_name = downtown_data.loc[5, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, neighbourhood_lat, neighbourhood_lng))


Latitude and longitude values of Central Bay Street are 43.65609000000006, -79.38492999999994.


In [95]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_lat, neighbourhood_lng, VERSION, radius, LIMIT)

In [96]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb012cf4467db32a33481c0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 76,
  'suggestedBounds': {'ne': {'lat': 43.660590004500065,
    'lng': -79.37872181095085},
   'sw': {'lat': 43.65158999550006, 'lng': -79.39113818904903}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ab2b0b9f964a520e56b20e3',
       'name': 'The Queen and Beaver Public House',
       'location': {'address': '35 Elm St.',
        'crossStreet': 'at Bay St.',
        'lat': 43.65747228208784,
        'lng': -79.38352412327917,

In [97]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [99]:
import json 
from pandas.io.json import json_normalize 

venues = results['response']['groups'][0]['items']  
nearby_venues = pd.json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,The Queen and Beaver Public House,Gastropub,43.657472,-79.383524
1,The Elm Tree Restaurant,Modern European Restaurant,43.657397,-79.383761
2,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
3,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
4,Poke Guys,Poke Place,43.654895,-79.385052
5,MUJI,Miscellaneous Shop,43.656024,-79.383284
6,Downtown Toronto,Neighborhood,43.653232,-79.385296
7,Japango,Sushi Restaurant,43.655268,-79.385165
8,Hailed Coffee,Coffee Shop,43.658833,-79.383684
9,Silver Snail Comics,Comic Shop,43.657031,-79.381403


In [100]:
print('{} venues were returned by Foursquare'.format(nearby_venues.shape[0]))

76 venues were returned by Foursquare


In [104]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
       
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues for each neighbourhood in Downtown Toronto

In [105]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [106]:
downtown_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Group dataframe by Neighbourhoods and count number of venues

In [109]:
downtown_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,60,60,60,60,60,60
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",76,76,76,76,76,76
Central Bay Street,76,76,76,76,76,76
Christie,11,11,11,11,11,11
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",58,58,58,58,58,58


In [112]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 195 uniques categories.


In [114]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")


downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 


fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped.head(10)

,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.016667
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,...,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.013158
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.013158,0.013158,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013158,0.013158,0.013158,0.000000,0.0,0.0,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
4,Church and Wellesley,0.012658,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,0.0,0.0,0.012658
5,"Commerce Court, Victoria Hotel",0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.0,0.010000
6,"First Canadian Place, Underground city",0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,...,0.010000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.0,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.0,0.0,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.017241,0.017241,0.000000,0.000000,0.000000,0.000000,0.017241,0.0,0.0,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.000000,0.000000,...,0.000000,0.052632,0.000000,0.000000,0.035088,0.017544,0.000000,0.0,0.0,0.000000


### Get top 5 Venues per Neighbourhood

In [117]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [124]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']


columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market
1,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,French Restaurant,Bar
2,Central Bay Street,Coffee Shop,Clothing Store,Bookstore,Restaurant,Bubble Tea Shop
3,Christie,Café,Grocery Store,Playground,Italian Restaurant,Candy Store
4,Church and Wellesley,Coffee Shop,Gay Bar,Restaurant,Japanese Restaurant,Café
5,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Restaurant,Café,Gym
6,"First Canadian Place, Underground city",Coffee Shop,Hotel,Café,Restaurant,Gym
7,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Hotel,Plaza,Restaurant,Japanese Restaurant
9,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Coffee Shop,Bar,Vegetarian / Vegan Restaurant


### Run KMeans to cluster neighbourhoods into 5 clusters

In [141]:
kclusters = 5
downtown_grouped_cluster = downtown_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(downtown_grouped_cluster)

kmeans.labels_[:10]

array([1, 1, 1, 2, 1, 1, 1, 1, 1, 1], dtype=int32)

In [142]:
downtown_merged = downtown_data

downtown_merged['Cluster Labels'] = kmeans.labels_

downtown_merged = downtown_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,1,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Italian Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,1,Coffee Shop,Sandwich Place,Mediterranean Restaurant,Theater,Falafel Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,2,Coffee Shop,Cocktail Bar,Cosmetics Shop,Restaurant,Gastropub
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,1,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market


### Display clusters in Map

In [143]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location = [lat, lng], zoom_start=13)


x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

### Examine each of the 5 clusters

In [150]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,Downtown Toronto,0,Coffee Shop,Hotel,Restaurant,Café,Gym


In [145]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Italian Restaurant
1,Downtown Toronto,1,Coffee Shop,Sandwich Place,Mediterranean Restaurant,Theater,Falafel Restaurant
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop
4,Downtown Toronto,1,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market
5,Downtown Toronto,1,Coffee Shop,Clothing Store,Bookstore,Restaurant,Bubble Tea Shop
6,Downtown Toronto,1,Café,Grocery Store,Playground,Italian Restaurant,Candy Store
7,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Gym
8,Downtown Toronto,1,Coffee Shop,Hotel,Plaza,Restaurant,Japanese Restaurant
9,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Salad Place
12,Downtown Toronto,1,Café,Mexican Restaurant,Coffee Shop,Bar,Vegetarian / Vegan Restaurant


In [146]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Cosmetics Shop,Restaurant,Gastropub


In [147]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,Downtown Toronto,3,Italian Restaurant,Coffee Shop,Café,French Restaurant,Bar


In [148]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,Downtown Toronto,4,Café,Coffee Shop,Bakery,Bookstore,Gym
